Travaillons la donnée pour constituer un fichier JSON de la forme : 

{  
  "id": "<UUID ou concat source+num>",  
  "text": "<contenu du passage>",  
  "metadata": {  
    "source": "<nom de la CSV ou type de document>",  
    "date": "YYYY-MM-DD",  
    "section": "<titre de la section>",  
    // autres champs utiles…  
  }  
}

Il faudra également chunker :
 
Découpage en passages de ~500 mots (ou ~2 000 caractères) pour un meilleur score de similarité.
	•	Vous pouvez chunker par paragraphes puis regrouper si trop courts
	•	Conserver les métadonnées à chaque chunk (idem date, section…)

In [1]:
import pandas as pd
import textwrap
import json


path = './fiscal_data_process_20232024/'

### Fonctions générales - Chunking & JSON

In [2]:
def optimized_chunk_dataframe(df: pd.DataFrame, column: str, chunk_size: int = 500) -> pd.DataFrame:
    """
    Optimized version: utilise apply + explode pour découper la colonne textuelle en chunks
    et dupliquer les autres colonnes de manière vectorisée.

    Args:
        df (pd.DataFrame): DataFrame d'entrée.
        column (str): Nom de la colonne à chunker.
        chunk_size (int, optional): Taille maximale des chunks en caractères. Par défaut 500.

    Returns:
        pd.DataFrame: DataFrame transformé avec colonnes dupliquées et colonne chunkée.
    """
    # 1. Créer une nouvelle colonne 'chunks' contenant la liste des morceaux
    df = df.copy()
    df['chunks'] = df[column].apply(lambda txt: textwrap.wrap(txt, width=chunk_size))
    
    # 2. Exploser cette colonne pour obtenir une ligne par chunk
    df_expanded = df.explode('chunks').reset_index(drop=True)
    
    # 3. Remplacer l'ancienne colonne par les chunks et supprimer la colonne temporaire
    df_expanded[column] = df_expanded['chunks']
    return df_expanded.drop(columns=['chunks'])



In [3]:

def df_to_jsonl(df: pd.DataFrame, id_col: str, text_col: str, metadata_cols: list, output_path: str = None) -> list:
    """
    Converts a DataFrame into a list of JSON Lines (JSONL) strings with the structure:
    {
      "id": "<id value>",
      "text": "<text value>",
      "metadata": {
        "<meta1>": "<value1>",
        "<meta2>": "<value2>",
        ...
      }
    }
    
    Args:
        df (pd.DataFrame): Input DataFrame.
        id_col (str): Name of the column to use as the document ID.
        text_col (str): Name of the column containing the text content.
        metadata_cols (list): List of column names to include in the metadata.
        output_path (str, optional): If provided, writes JSONL to this file path.
    
    Returns:
        list: List of JSONL strings.
    """
    jsonl_lines = []
    
    for _, row in df.iterrows():
        record = {
            "id": str(row[id_col]),
            "text": row[text_col],
            "metadata": {col: row[col] for col in metadata_cols}
        }
        jsonl_line = json.dumps(record, ensure_ascii=False)
        jsonl_lines.append(jsonl_line)
    
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            for line in jsonl_lines:
                f.write(line + '\n')
    
    return jsonl_lines


### Constitution du JSON general

In [8]:
### BOFIP
df_bofip = pd.read_csv(path + 'Bofip_2023_2024.csv', sep =";")

df_bofip_chunked = optimized_chunk_dataframe(df_bofip, 'contenu', chunk_size=500)
json_bofip = df_to_jsonl(df_bofip_chunked, id_col='identifiant_juridique', text_col='contenu',
                        metadata_cols=['division', 'debut_de_validite', 'serie', 'titre'],
                        output_path='./json_chunk/bofip.jsonl')

In [6]:
### CGI
df_cgi = pd.read_csv(path + 'CGI_2023_2024.csv', sep =";")

In [10]:
df_cgi_chunked = optimized_chunk_dataframe(df_cgi, 'content', chunk_size=500)
json_cgi = df_to_jsonl(df_cgi_chunked, id_col='id', text_col='content',
                        metadata_cols=['num', 'etat',  'pathTitle','cid', 'section_title', 'parent_title'],
                        output_path='./json_chunk/cgi.jsonl')

In [11]:
### Jurisprudence
df_juris = pd.read_csv(path + 'Juris_filter_2023_2024.csv', sep =";")

In [14]:
df_juris_chunked = optimized_chunk_dataframe(df_juris, 'texte_integral', chunk_size=500)
json_juris = df_to_jsonl(df_juris_chunked, id_col='id', text_col='texte_integral',
                        metadata_cols=['date_maj', 'code_juridiction', 'juridiction','date_decision',  'type_decision', 'type_recours',
                                          'solution', 'date_audience', 'formation_jugement'],
                        output_path='./json_chunk/juris.jsonl')

In [15]:
### LPF
df_lpf = pd.read_csv(path + 'LPF_2023_2024.csv', sep =";")

In [18]:
df_lpf_chunked = optimized_chunk_dataframe(df_lpf, 'content', chunk_size=500)
json_lpf = df_to_jsonl(df_lpf_chunked, id_col='id', text_col='content',
                        metadata_cols=[ 'num', 'etat', 'pathTitle', 'section_title', 'parent_title'],
                        output_path='./json_chunk/lpf.jsonl')

In [19]:
### QA AN
df_qa = pd.read_csv(path + 'QA_AN_2023_2024.csv', sep =";")

In [25]:
df_qa['QA'] = " Question : " + df_qa['texte_question'] + "\n Reponse : " + df_qa['texte_reponse'] 

In [28]:
df_qa_chunked = optimized_chunk_dataframe(df_qa, 'QA', chunk_size=500)
json_qa = df_to_jsonl(df_qa_chunked, id_col='uid', text_col='QA',
                        metadata_cols=['legislature', 'rubrique', 'analyse','auteur_groupe', 'ministere', 'date_question', 'date_reponse', 'cloture_date','date_publication'],
                        output_path='./json_chunk/qa.jsonl')

In [29]:
### GLOBAL
def merge_jsonl_lists(lists_of_lines: list, output_path: str = None) -> list:
    """
    Fusionne plusieurs listes de lignes JSONL en une seule liste.

    Args:
        lists_of_lines (list of list of str): Listes de chaînes JSONL.
        output_path (str, optional): Si fourni, écrit la liste fusionnée dans ce fichier.

    Returns:
        list of str: Liste fusionnée de lignes JSONL.
    """
    merged = []
    for lines in lists_of_lines:
        merged.extend(lines)

    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            for line in merged:
                # S’assure qu’il y a un seul passage à la ligne par enregistrement
                f.write(line.rstrip('\n') + '\n')

    return merged


In [30]:
all_lines = merge_jsonl_lists([json_bofip, json_cgi, json_juris, json_lpf, json_qa], output_path='./json_chunk/all_data.jsonl')

In [33]:
len(all_lines)

778750

In [35]:
len(all_lines)

778750